In [1]:
import pandas as pd
import numpy as np
import os
import re
import ast
%config PPMagics.autolimit=0
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
!pip install nltk -i https://dtr-vip.ccg13.slc.paypalinc.com/artifactory/api/pypi/python/simple
!pip install ftfy -i https://dtr-vip.ccg13.slc.paypalinc.com/artifactory/api/pypi/python/simple
!pip install emoji -i https://dtr-vip.ccg13.slc.paypalinc.com/artifactory/api/pypi/python/simple
!pip install bs4
!pip install html5lib
!pip install lxml

Looking in indexes: https://dtr-vip.ccg13.slc.paypalinc.com/artifactory/api/pypi/python/simple

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://dtr-vip.ccg13.slc.paypalinc.com/artifactory/api/pypi/python/simple

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://dtr-vip.ccg13.slc.paypalinc.com/artifactory/api/pypi/python/simple
  Using cached https://dtr-vip.ccg13.slc.paypalinc.com/artifactory/api/pypi/python/packages/packages/96/c6/0114b2040a96561fd1b44c75df749bbd3c898bf8047fb5ce8d7590d2dee6/emoji-2.8.0-py2.py3-none-any.whl (358 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://dtr-vip.ccg13.slc.paypalinc.com/artifactory/api/pypi/python/simple

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] 

In [3]:
import pandas as pd
import string, unicodedata, emoji
import regex as re
import warnings
import re
import nltk

from bs4 import BeautifulSoup
from nltk import sent_tokenize
from ftfy import fix_text
from tqdm import  tqdm

warnings.filterwarnings("ignore", category=UserWarning, module='bs4')
tqdm.pandas()

In [5]:

transcripts_features_WK36_Validated_stage1=pd.read_csv('dailogues_week2crypto_validated.csv')

In [7]:
#transcripts_products_stage1.shape,transcripts_features_stage1.shape
transcripts_features_WK36_Validated_stage1.shape

(150, 5)

## CHAT

In [8]:
_all_letters = string.ascii_letters + string.digits + string.punctuation + string.whitespace

_flag1 = re.MULTILINE | re.DOTALL
_flag2 = re.MULTILINE | re.DOTALL | re.IGNORECASE

_placeholders = ['<transaction>', '<hashtag>', '<allcaps>', '<url>', '<smile>', '<lolface>', '<sadface>', '<neutralface>',
                 '<heart>', '<repeat>', '<elong>', '<money>', '<date>', '<time>', '<number>', '<email>']

_emoji_vocab = ['{:expressionless_face:}', '{:clown_face:}', '{:unamused_face:}', '{:face_with_tears_of_joy:}', '{:exploding_head:}',
               '{:folded_hands:}', '{:smiling_face_with_smiling_eyes:}', '{:man_facepalming_medium_skin_tone:}', '{:woozy_face:}', '{:pleading_face:}',
               '{:face_with_symbols_on_mouth:}', '{:fire:}', '{:loudly_crying_face:}', '{:face_with_rolling_eyes:}', '{:winking_face:}', '{:pouting_face:}',
               '{:face_blowing_a_kiss:}', '{:thinking_face:}', '{:sad_but_relieved_face:}', '{:woman_facepalming:}', '{:rolling_on_the_floor_laughing:}',
               '{:folded_hands_light_skin_tone:}', '{:grinning_face:}', '{:pensive_face:}', '{:', '{:angry_face:}', '{:thumbs_up:}', '{:copyright:}',
               '{:credit_card:}', '{:downcast_face_with_sweat:}', '{:slightly_smiling_face:}', '{:raised_hand_dark_skin_tone:}', '{:confused_face:}',
               '{:face_vomiting:}', '{:angry_face_with_horns:}', '{:persevering_face:}', '{:crying_face:}', '{:musical_notes:}', '{:nerd_face:}', '{:flushed_face:}',
               '{:heart_suit:}', '{:disappointed_face:}', '{:woman_facepalming_light_skin_tone:}', '{:relieved_face:}', '{:man_shrugging:}', '{:black_heart:}',
               '{:red_heart:}', '{:saudi_arabia:}', '{:face_with_steam_from_nose:}', '{:neutral_face:}', '{:thumbs_down:}', '{:man_shrugging_light_skin_tone:}',
               '{:robot_face:}', '{:backhand_index_pointing_down:}', '{:person_facepalming:}', '{:person_shrugging:}', '{:frowning_face:}', '{:waving_hand:}',
               '{:smiling_face:}', '{:beaming_face_with_smiling_eyes:}', '{:weary_face:}', '{:thumbs_up_light_skin_tone:}', '{:united_states:}',
               '{:woman_shrugging_light_skin_tone:}', '{:wastebasket:}', '{:woman_shrugging_medium_skin_tone:}', '{:anguished_face:}', '{:hundred_points:}',
               '{:flexed_biceps:}', '{:graduation_cap:}', '{:united_kingdom:}', '{:france:}', '{:man_shrugging_medium_skin_tone:}', '{:registered:}',
               '{:sleeping_face:}', '{:ballot_box_with_check:}', '{:clapping_hands:}', '{:two_hearts:}', '{:sleepy_face:}', '{:laptop_computer:}',
               '{:slightly_frowning_face:}', '{:worried_face:}', '{:face_screaming_in_fear:}', '{:smirking_face:}', '{:broken_heart:}', '{:index_pointing_up:}',
               '{:middle_finger:}', '{:grinning_face_with_sweat:}', '{:folded_hands_medium_skin_tone:}', '{:money_bag:}', '{:blue_heart:}',
               '{:smiling_face_with_sunglasses:}', '{:beating_heart:}', '{:backhand_index_pointing_right:}', '{:money_with_wings:}', '{:nauseated_face:}',
               '{:face_with_medical_mask:}', '{:heavy_dollar_sign:}', '{:dollar_banknote:}', '{:party_popper:}', '{:sparkles:}', '{:man_facepalming:}',
               '{:tired_face:}', '{:confounded_face:}', '{:kissing_face:}', '{:backhand_index_pointing_up_light_skin_tone:}', '{:tulip:}', '{:hugging_face:}',
               '{:squinting_face_with_tongue:}', '{:shushing_face:}', '{:globe_showing_americas:}', '{:china:}', '{:eyes:}', '{:grimacing_face:}', '{:high_voltage:}',
               '{:rocket:}', '{:cherry_blossom:}', '{:blossom:}', '{:bomb:}', '{:rose:}', '{:socks:}', '{:green_apple:}', '{:face_with_monocle:}',
               '{:woman_shrugging:}', '{:skull:}', '{:man_facepalming_light_skin_tone:}', '{:woman_facepalming_medium_skin_tone:}', '{:victory_hand:}',
               '{:grinning_face_with_smiling_eyes:}', '{:thumbs_down_medium_skin_tone:}', '{:backhand_index_pointing_up:}', '{:raised_fist_medium_skin_tone:}',
               '{:sun_with_face:}', '{:purple_heart:}', '{:thumbs_down_light_skin_tone:}', '{:bottle_with_popping_cork:}', '{:pile_of_poo:}', '{:fearful_face:}',
               '{:parrot:}', '{:dove:}', '{:index_pointing_up_light_skin_tone:}', '{:face_with_raised_eyebrow:}', '{:trade_mark:}',
               '{:kissing_face_with_closed_eyes:}', '{:night_with_stars:}', '{:outbox_tray:}', '{:raising_hands_light_skin_tone:}', '{:female_sign:}',
               '{:green_heart:}', '{:grinning_squinting_face:}', '{:scotland:}', '{:smiling_face_with_halo:}', '{:hourglass_done:}',
               '{:man_raising_hand_light_skin_tone:}', '{:cooked_rice:}', '{:nepal:}', '{:smiling_face_with_<number>_hearts:}', '{:alien:}',
               '{:anxious_face_with_sweat:}', '{:lying_face:}', '{:crossed_fingers:}', '{:middle_finger_dark_skin_tone:}', '{:unlocked:}',
               '{:person_shrugging_light_skin_tone:}', '{:face_with_open_mouth:}', '{:partying_face:}', '{:yellow_heart:}', '{:handshake:}', '{:hushed_face:}',
               '{:winking_face_with_tongue:}', '{:turkey:}', '{:hedgehog:}', '{:cactus:}', '{:woman_with_headscarf:}', '{:books:}', '{:bouquet:}', '{:memo:}',
               '{:right_arrow_curving_down:}', '{:grinning_face_with_big_eyes:}', '{:germany:}', '{:jamaica:}', '{:thumbs_up_medium_skin_tone:}',
               '{:person_raising_hand:}', '{:raising_hands:}', '{:index_pointing_up_dark_skin_tone:}', '{:stop_sign:}', '{:telephone:}',
               '{:frowning_face_with_open_mouth:}', '{:chile:}', '{:person_facepalming_medium_skin_tone:}', '{:fox_face:}', '{:sneezing_face:}',
               '{:folded_hands_dark_skin_tone:}', '{:star_of_david:}', '{:sunset:}', '{:myanmar_(burma):}', '{:victory_hand_light_skin_tone:}', '{:kiss_mark:}',
               '{:white_heavy_check_mark:}', '{:house_with_garden:}', '{:face_without_mouth:}', '{:snowflake:}', '{:middle_finger_light_skin_tone:}',
               '{:sparkling_heart:}', '{:face_with_thermometer:}', '{:ireland:}', '{:musical_score:}', '{:information:}', '{:llama:}', '{:prohibited:}',
               '{:indonesia:}', '{:heart_with_arrow:}', '{:hot_face:}', '{:kissing_face_with_smiling_eyes:}', '{:nail_polish:}', '{:shopping_cart:}',
               '{:thumbs_up_dark_skin_tone:}', '{:tumbler_glass:}', '{:white_medium_star:}', '{:exclamation_question_mark:}', '{:ox:}', '{:double_exclamation_mark:}',
               '{:kenya:}', '{:speaking_head:}', '{:sign_of_the_horns_light_skin_tone:}', '{:face_with_hand_over_mouth:}', '{:comet:}', '{:cowboy_hat_face:}',
               '{:warning:}', '{:bread:}', '{:melon:}', '{:peach:}', '{:roasted_sweet_potato:}', '{:ear_of_corn:}', '{:pineapple:}', '{:pizza:}', '{:tomato:}',
               '{:watermelon:}', '{:meat_on_bone:}', '{:horse_racing:}', '{:oncoming_taxi:}', '{:oncoming_automobile:}', '{:articulated_lorry:}', '{:oncoming_bus:}',
               '{:ophiuchus:}', '{:capricorn:}', '{:aries:}', '{:sagittarius:}', '{:spain:}', '{:colombia:}', '{:israel:}', '{:direct_hit:}', '{:call_me_hand:}',
               '{:malaysia:}', '{:dizzy_face:}', '{:zany_face:}', '{:right_arrow:}', '{:alarm_clock:}', '{:red_circle:}', '{:question_mark:}',
               '{:waving_hand_light_skin_tone:}', '{:mobile_phone:}', '{:drooling_face:}', '{:growing_heart:}', '{:bell:}', '{:world_map:}', '{:person_fencing:}',
               '{:boxing_glove:}', '{:snowboarder:}', '{:<number>rd_place_medal:}', '{:sports_medal:}', '{:<number>nd_place_medal:}', '{:skier:}',
               '{:<number>st_place_medal:}', '{:performing_arts:}', '{:ticket:}', '{:martial_arts_uniform:}', '{:raised_hand_medium_skin_tone:}',
               '{:sign_of_the_horns:}', '{:backhand_index_pointing_up_dark_skin_tone:}', '{:backhand_index_pointing_left:}', '{:telephone_receiver:}', '{:rainbow:}',
               '{:person_tipping_hand:}', '{:clapping_hands_light_skin_tone:}', '{:hibiscus:}', '{:victory_hand_medium_skin_tone:}', '{:snake:}',
               '{:astonished_face:}', '{:envelope:}', '{:one_oclock:}', '{:love_letter:}', '{:skull_and_crossbones:}', '{:toilet:}', '{:australia:}', '{:italy:}',
               '{:greece:}', '{:switzerland:}', '{:up_arrow:}', '{:right_arrow_curving_up:}', '{:waving_hand_medium_skin_tone:}', '{:thought_balloon:}',
               '{:police_car_light:}', '{:person_playing_handball:}', '{:thumbs_down_dark_skin_tone:}', '{:desktop_computer:}', '{:keycap_digit_nine:}',
               '{:heavy_plus_sign:}', '{:face_savoring_food:}', '{:new_moon_face:}', '{:cross_mark:}', '{:heavy_check_mark:}', '{:mouse_face:}',
               '{:keycap_digit_one:}', '{:keycap_digit_two:}', '{:locked:}', '{:bank:}', '{:mobile_phone_with_arrow:}', '{:bookmark_tabs:}', '{:video_game:}',
               '{:newspaper:}', '{:light_bulb:}', '{:revolving_hearts:}', '{:birthday_cake:}', '{:anger_symbol:}', '{:loudly_crying_face:}',
               '{:japanese_bargain_button:}', '{:japanese_no_vacancy_button:}', '{:flexed_biceps_light_skin_tone:}',
               '{:backhand_index_pointing_down_light_skin_tone:}', '{:cookie:}', '{:crying_cat_face:}', '{:wind_face:}', '{:balloon:}',
               '{:clapping_hands_medium_skin_tone:}', '{:person_bowing:}', '{:bust_in_silhouette:}', '{:heart_with_ribbon:}', '{:sun:}', '{:white_small_square:}',
               '{:backhand_index_pointing_right_medium_skin_tone:}', '{:pakistan:}', '{:volcano:}', '{:hot_beverage:}', '{:face_with_tongue:}', '{:uganda:}',
               '{:oncoming_fist:}', '{:heavy_heart_exclamation:}', '{:collision:}', '{:goblin:}', '{:ogre:}', '{:lion_face:}', '{:pig_nose:}',
               '{:raising_hands_dark_skin_tone:}', '{:pound_banknote:}', '{:euro_banknote:}', '{:clapping_hands_dark_skin_tone:}', '{:woman_artist:}',
               '{:man_artist:}', '{:baby_symbol:}', '{:carp_streamer:}', '{:four_leaf_clover:}', '{:algeria:}', '{:argentina:}', '{:canada:}', '{:repeat_button:}',
               '{:oncoming_fist_light_skin_tone:}', '{:philippines:}', '{:vulcan_salute_light_skin_tone:}', '{:glowing_star:}', '{:middle_finger_medium_skin_tone:}',
               '{:headphone:}', '{:dog:}', '{:popcorn:}', '{:film_frames:}', '{:man_dancing:}', '{:swan:}', '{:moai:}', '{:person_rowing_boat:}',
               '{:call_me_hand_medium_skin_tone:}', '{:litter_in_bin_sign:}', '{:raised_fist:}', '{:weary_cat_face:}', '{:exclamation_mark:}', '{:crown:}',
               '{:small_orange_diamond:}', '{:red_triangle_pointed_up:}', '{:backhand_index_pointing_right_dark_skin_tone:}', '{:cat_face_with_tears_of_joy:}',
               '{:snowman_without_snow:}', '{:snowman:}', '{:turtle:}', '{:broom:}', '{:waving_hand_dark_skin_tone:}', '{:fish:}', '{:maple_leaf:}',
               '{:dashing_away:}', '{:trophy:}', '{:white_question_mark:}', '{:backhand_index_pointing_right_light_skin_tone:}', '{:rat:}', '{:butterfly:}',
               '{:battery:}', '{:no_entry:}', '{:man_dancing_light_skin_tone:}', '{:sweat_droplets:}', '{:oman:}', '{:man_facepalming_dark_skin_tone:}',
               '{:paintbrush:}', '{:mosquito:}', '{:lady_beetle:}', '{:cricket:}' '{:spider:}', '{:stopwatch:}', '{:orange_heart:}',
               '{:person_facepalming_light_skin_tone:}', '{:saxophone:}', '{:trumpet:}', '{:b_button_(blood_type):}', '{:o_button_(blood_type):}',
               '{:envelope_with_arrow:}', '{:smiling_face_with_horns:}', '{:malta:}', '{:sri_lanka:}', '{:oncoming_police_car:}', '{:two_oclock:}',
               '{:woman_biking:}', '{:crossed_fingers_light_skin_tone:}', '{:baby_angel:}', '{:woman_raising_hand_light_skin_tone:}', '{:reminder_ribbon:}',
               '{:military_medal:}', '{:heavy_large_circle:}', '{:south_africa:}', '{:down_arrow:}', '{:backhand_index_pointing_up_medium_skin_tone:}', '{:england:}',
               '{:oncoming_fist_medium_skin_tone:}', '{:raised_hand:}', '{:nigeria:}', '{:horse:}', '{:pouting_cat_face:}', '{:hot_pepper:}', '{:person_frowning:}',
               '{:man_tipping_hand_light_skin_tone:}', '{:man_detective:}', '{:bangladesh:}', '{:monkey:}', '{:two_men_holding_hands:}', '{:camera_with_flash:}',
               '{:pistol:}', '{:play_button:}', '{:red_triangle_pointed_down:}', '{:palestinian_territories:}', '{:regional_indicator_symbol_letter_p:}',
               '{:superhero:}', '{:male_sign:}', '{:raised_fist_dark_skin_tone:}', '{:cat_face_with_wry_smile:}', '{:eye:}', '{:cambodia:}', '{:clinking_beer_mugs:}',
               '{:goat:}', '{:link:}', '{:sunflower:}', '{:iraq:}', '{:baseball:}', '{:fireworks:}', '{:no_mobile_phones:}', '{:watch:}', '{:locked_with_key:}',
               '{:speaker_high_volume:}', '{:ghana:}', '{:ant:}', '{:wrapped_gift:}', '{:foot:}', '{:gorilla:}', '{:new_moon:}', '{:receipt:}', '{:zimbabwe:}',
               '{:floppy_disk:}', '{:crossed_fingers_medium_skin_tone:}', '{:woman_dancing:}', '{:ear:}', '{:wheelchair_symbol:}', '{:man_walking_medium_skin_tone:}',
               '{:grinning_cat_face_with_smiling_eyes:}', '{:peace_symbol:}', '{:person_walking:}', '{:confetti_ball:}', '{:television:}', '{:kitchen_knife:}',
               '{:water_wave:}', '{:trinidad_&_tobago:}', '{:cloud:}', '{:senegal:}', '{:hand_with_fingers_splayed_light_skin_tone:}',
               '{:person_tipping_hand_light_skin_tone:}', '{:croatia:}', '{:grinning_cat_face:}', '{:hatching_chick:}', '{:raised_back_of_hand:}',
               '{:chequered_flag:}']
emoji_vocab = list(set(_emoji_vocab))

special_tokens = list(set(_emoji_vocab + _placeholders))

def _re_sub(_pattern, _repl, _text, _flag):
    return re.sub(_pattern, _repl, _text, flags=_flag)


def _glove_preprocessing(_text):
    # _FLAGS = re.MULTILINE | re.DOTALL
    def hashtag(_text):
        _text = _text.group()
        hashtag_body = _text[1:]
        if hashtag_body.isupper():
            result = " <hashtag> {} <allcaps> ".format(hashtag_body)
        else:
            result = " ".join([" <hashtag> "] + re.split(r"(?=[A-Z])", hashtag_body, flags=_flag1))
        return result

    def allcaps(_text):
        _text = _text.group()
        return _text.lower() + " <allcaps> "

    eyes = r"[8:=;]"
    nose = r"['`\-]?"
    _text = _re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", " <url> ", _text, _flag2)
    _text = _re_sub(r"/", " / ", _text, _flag2)
    _text = _re_sub(r"@\w+", " <user> ", _text, _flag2)
    _text = _re_sub(r"{}[\s+]?{}[\s+]?[)dD](\b)+|[)dD]+{}{}(\b)".format(eyes, nose, nose, eyes), " <smile> ", _text, _flag2)
    _text = _re_sub(r"{}[\s+]?{}p+(\b)".format(eyes, nose), " <lolface> ", _text, _flag2)
    _text = _re_sub(r"{}[\s+]?{}\((\b)+|\)+{}[\s+]?{}(\b)".format(eyes, nose, nose, eyes), " <sadface> ", _text, _flag2)
    _text = _re_sub(r"{}[\s+]?{}[\/|l*](\b)".format(eyes, nose), " <neutralface> ", _text, _flag2)
    _text = _re_sub(r"<3(\b)", " <heart> ", _text, _flag2)
    try:
        _text = _re_sub(r"#\S+", hashtag, _text, _flag1)
    except:
        pass
    _text = _re_sub(r"(['!?.]){2,}", r"\1 <repeat> ", _text, _flag2)
    _text = _re_sub(r"(\S*?)([a-z\.\-\+!@#$%\^&*()~,\?\:|])\2{3,}", r"\1\2 <elong> ", _text, _flag2)
    _text = _re_sub(r"([A-Z]){2,}[^'\*]", allcaps, _text, _flag1)
    return _text


def _unicode_emoji_process(matched):
    emo = matched.group()
    emo_al = emoji.demojize(emo)
    return '{' + emo_al + '}' + ' '


def _unicode_to_ascii(s):
    s = str(s)
    return ''.join(
        c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn' and c in _all_letters
    )


def _strip_hmtl_tags(s):
    _removed = BeautifulSoup(s, 'html.parser').get_text().strip()
    return _removed


def _replace_currency(_text):
    _text = _re_sub(r'(^|\b|\s)[$€£¥]{1}[0-9,]+(?:\.[0-9]+)?\b', " <money> ", _text, _flag2)
    return _text


def _process_transaction_text_GB(_text):
    if 'transactionText' in _text:
        _text = _re_sub(r'transactionText[:\s]{1}[*|0-9A-Z|£\s.,]+', " <transaction> ", _text, _flag2)

    return _text


def _replace_datetime(_text):
    # dd-mm-yyyy
    date_patten_1 = r'\b\d\s*(?:Jan|January|Feb|February|Mar|March|Apr|April|May|May|Jun|June|Jul|July|Aug|August|Sep|September|Oct|October|Nov|November|Dec|December)\s*\d{4}\b'
    date_patten_2 = r'\d{1,2}\s*[/-]\s*\d{1,2}\s*[/-]\s*\d{2,4}' # 01/11/2011 01-02-2011 01/11/22
    date_patten_3 = r'\d{1,2}\s*[/-]\s*\d{1,2}\s*[/-]\s*\d{2,4}' # 01/11/2011 01-02-2011 01/11/22
    time_patten = r'\b\d{1,2}\s*\:\d{1,2}\s*(?:am|pm)?'
    _text = _re_sub(date_patten_1, " <date> ", _text, _flag2)
    _text = _re_sub(date_patten_2, " <date> ", _text, _flag2)
    _text = _re_sub(time_patten, " <time> ", _text, _flag2)
    return _text


def _replace_numbers(_text):
    _num_patten1 = r'\d+\s*[-/,.]{1}\s*\d+'
    _num_patten2 = r'\d+'
    _text = _re_sub(_num_patten1, " <number> ", _text, _flag2)# 23-23 12/23 1,234 0.23 2323
    _text = _re_sub(_num_patten2, " <number> ", _text, _flag2)# 23-23 12/23 1,234 0.23 2323
    return _text


def _replace_email(_text):
    # _email_patten = r'(?:(\s|$|\b))[a-zA-Z0-9_\.]+@[a-zA-Z0-9.]+(com)'
    _email_patten = r'\S+@\S+'
    _text = _re_sub(_email_patten, " <email> ", _text, _flag2)
    return _text


def preprocess_text(_input):
    _content = _strip_hmtl_tags(_input)
    _content = _process_transaction_text_GB(_content)
    _content = _replace_email(_content)
    _content = _glove_preprocessing(_content)
    _content = emoji.demojize(_content, delimiters=("{:", ":}"))
    _content = _replace_currency(_content)
    _content = _replace_datetime(_content)
    _content = _replace_numbers(_content)
    _content = _unicode_to_ascii(_content)
    return _content

In [9]:
CLS_TOKEN = "<s> "
SEP_TOKEN = "</s>"
def process_raw_text(_input_text):
    '''
    this function helps in preprocessing the text - same from identification pipeline
    input_data: str
    :return: str
    '''    
    def __text_process(_line):
        lst_sentence = sent_tokenize(fix_text(_line))
        lst_sentence_prc = map(lambda sen: preprocess_text(sen), lst_sentence)
        lst_sentence_prc = [sen.strip() for sen in lst_sentence_prc]
        lst_sentence_prc = [sent for sent in lst_sentence_prc if len(sent) > 0]
        return ' '.join(lst_sentence_prc)

    def __get_sender(_sender_str):
        _sender = ''
        if _sender_str.lower() == 'customer':
            _sender = 'CLIENT'
        elif _sender_str.lower() == 'agent':
            _sender = 'AGENT'
        elif _sender_str.lower() == 'bot' or _sender_str.lower() == 'external' or _sender_str.lower() == 'system':
            _sender = 'BOT'
        return _sender

    lines = _input_text.split('\t')
    lines = [l_.strip() for l_ in lines if l_.strip() != '']
    # Customer:, Agent:
    proc_lines = []
    for l_idx, l_ in enumerate(lines):
        # l_ = l_[14:] # remove pre-fixed time stamp
        _s_idx = l_.find(':')
        assert _s_idx > 0
        sender_str = l_[0: _s_idx].lower()
        content = l_[_s_idx+1:].strip()
        assert sender_str in ['customer', 'agent', 'bot', 'external', 'system','unknown','auto agent'], "The sender %s not found" %sender_str
        sender = __get_sender(sender_str)
        l_preprocessed = __text_process(content)
        if len(l_preprocessed.split()) > 0:
            proc_lines.append([sender, l_preprocessed])

    # combine same consecutive senders
    combined_posts = []
    if len(proc_lines) == 0:
        logging.info("Empty Input len of proc lines are zero")
        #pdb.set_trace()
        return CLS_TOKEN + " " + SEP_TOKEN

    try:
        proc_lines = CLS_TOKEN + ' '.join(list(map(lambda line: '{}: {} {}'.format(line[0], ' '.join(line[1].split()), SEP_TOKEN),proc_lines)))
    except:
        return "empty database"
    return proc_lines

In [10]:
transcripts_features_WK36_Validated_stage1_chat=transcripts_features_WK36_Validated_stage1[transcripts_features_WK36_Validated_stage1['media_type']=='chat'].reset_index(drop=True)
transcripts_features_WK36_Validated_stage1_chat.shape

(70, 5)

In [11]:
transcripts_features_WK36_Validated_stage1_chat['all_utterances_proc'] = transcripts_features_WK36_Validated_stage1_chat['all_utterances']
transcripts_features_WK36_Validated_stage1_chat['all_utterances_proc'] = transcripts_features_WK36_Validated_stage1_chat['all_utterances_proc'].progress_apply(process_raw_text)


100%|██████████| 70/70 [00:01<00:00, 35.20it/s]


In [1]:
#transcripts_features_WK34_Validated_stage1_chat.tail()

In [12]:
transcripts_features_WK36_Validated_stage1_chat.to_csv('transcripts_features_WK36_Validated_stage1_chat.csv',index=False)